In [151]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


In [152]:
def within_n_days(x, T, n = 7):
	# n = 7, 14, 30, 60, 90,
	##################################################
	# Filtering accroding to w
	##################################################
	if T == 0:
		# w = 0:使用3月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-03-01', '%Y-%m-%d')
	elif T == 1:
		# w = 1:使用2月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-02-01', '%Y-%m-%d')
	elif T == 2:
		# w = 2:使用1月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-01-01', '%Y-%m-%d')
	elif T == -1:
		# w = -1:使用4月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-04-01', '%Y-%m-%d')
	check_points = prediction_deadline - timedelta(n) # 往prediction_deadline前n天內
	x = x[ (x.date >= check_points )]
	return x

In [162]:
input_col = ['msno','date','num_25','num_100']
df = pd.read_csv('sampling_for_analyzing.csv', parse_dates = ['date'])[input_col]
df.head()

,msno,date,num_25,num_100
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-16,6.0,18.0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-17,3.0,28.0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-18,6.0,25.0
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-19,19.0,32.0
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-20,8.0,62.0


In [163]:
T = 0
user_logs = df
folder = 'trainW-'+ str(T)

In [159]:
# basic procedure
##################################################
#get_ratio
user_logs.loc[:,"num_25":"num_100"] = user_logs.loc[:,"num_25":"num_100"].div(user_logs.loc[:,"num_25":"num_100"].sum(axis=1), axis=0)
user_logs.rename(columns = {'num_25':'num_25_ratio', 'num_100':'num_100_ratio'}, inplace =True)
user_logs.info()
user_logs = user_logs.dropna()
# #==============================================================================
# print('reduce memory')
# #==============================================================================
utils.reduce_memory(user_logs)
df = user_logs

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
100%|██████████| 4/4 [00:00<00:00, 1246.54it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
msno             50000 non-null object
date             49966 non-null datetime64[ns]
num_25_ratio     49445 non-null float64
num_100_ratio    49445 non-null float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 1.5+ MB



/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49445 entries, 0 to 49999
Data columns (total 4 columns):
msno             49445 non-null object
date             49445 non-null datetime64[ns]
num_25_ratio     49445 non-null float32
num_100_ratio    49445 non-null float32
dtypes: datetime64[ns](1), float32(2), object(1)
memory usage: 1.5+ MB


In [164]:
##################################################
# basic procedure
##################################################
#get_ratio
user_logs.loc[:,"num_25":"num_100"] = user_logs.loc[:,"num_25":"num_100"].div(user_logs.loc[:,"num_25":"num_100"].sum(axis=1), axis=0)
user_logs.rename(columns = {'num_25':'num_25_ratio', 'num_100':'num_100_ratio'}, inplace =True)
user_logs.dropna(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(user_logs)
df = user_logs
#df = pd.merge(train, user_logs, on = 'msno', how = 'left')
del user_logs
gc.collect()
print ('shape of df:', df.shape)
##################################################
# All history
##################################################
#core1:num_25_ratio
print ('core1')
tbl = df.groupby('msno').num_25_ratio.mean().to_frame()
tbl.columns = ['num_25_ratio-mean']
tbl['num_25_ratio-min'] = df.groupby('msno').num_25_ratio.min()
tbl['num_25_ratio-max'] = df.groupby('msno').num_25_ratio.max()
tbl['num_25_ratio-median'] = df.groupby('msno').num_25_ratio.median()
tbl['num_25_ratio-std'] = df.groupby('msno').num_25_ratio.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_25_ratio.csv'.format(folder), index = False)
del tbl
gc.collect()
#core2:num_100_ratio
print ('core2')
tbl = df.groupby('msno').num_100_ratio.mean().to_frame()
tbl.columns = ['num_100_ratio-mean']
tbl['num_100_ratio-min'] = df.groupby('msno').num_100_ratio.min()
tbl['num_100_ratio-max'] = df.groupby('msno').num_100_ratio.max()
tbl['num_100_ratio-median'] = df.groupby('msno').num_100_ratio.median()
tbl['num_100_ratio-std'] = df.groupby('msno').num_100_ratio.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_100_ratio.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 7
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 7).reset_index(drop = True)
#core1:num_25_ratio
print ('core1')
tbl = df_.groupby('msno').num_25_ratio.mean().to_frame()
tbl.columns = ['num_25_ratio_during_t_7-mean']
tbl['num_25_ratio_during_t_7-min'] = df_.groupby('msno').num_25_ratio.min()
tbl['num_25_ratio_during_t_7-max'] = df_.groupby('msno').num_25_ratio.max()
tbl['num_25_ratio_during_t_7-median'] = df_.groupby('msno').num_25_ratio.median()
tbl['num_25_ratio_during_t_7-std'] = df_.groupby('msno').num_25_ratio.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_25_ratio_during_t_7.csv'.format(folder), index = False)
del tbl
gc.collect()
#core2:num_100_ratio
print ('core2')
tbl = df_.groupby('msno').num_100_ratio.mean().to_frame()
tbl.columns = ['num_repeated_songs_during_t_7-mean']
tbl['num_100_ratio_during_t_7-min'] = df_.groupby('msno').num_100_ratio.min()
tbl['num_100_ratio_during_t_7-max'] = df_.groupby('msno').num_100_ratio.max()
tbl['num_100_ratio_during_t_7-median'] = df_.groupby('msno').num_100_ratio.median()
tbl['num_100_ratio_during_t_7-std'] = df_.groupby('msno').num_100_ratio.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_100_ratio_during_t_7.csv'.format(folder), index = False)
del tbl
gc.collect()

##################################################
# n = 14
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 14).reset_index(drop = True)
#core1:completed_songs_ratio
print ('core1')
tbl = df_.groupby('msno').num_25_ratio.mean().to_frame()
tbl.columns = ['num_25_ratio_during_t_14-mean']
tbl['num_25_ratio_during_t_14-min'] = df_.groupby('msno').num_25_ratio.min()
tbl['num_25_ratio_during_t_14-max'] = df_.groupby('msno').num_25_ratio.max()
tbl['num_25_ratio_during_t_14-median'] = df_.groupby('msno').num_25_ratio.median()
tbl['num_25_ratio_during_t_14-std'] = df_.groupby('msno').num_25_ratio.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_25_ratio_during_t_14.csv'.format(folder), index = False)
del tbl
gc.collect()	
#core2:num_repeated_songs
print ('core2')
tbl = df_.groupby('msno').num_100_ratio.mean().to_frame()
tbl.columns = ['num_100_ratio_during_t_14-mean']
tbl['num_100_ratio_during_t_14-min'] = df_.groupby('msno').num_100_ratio.min()
tbl['num_100_ratio_during_t_14-max'] = df_.groupby('msno').num_100_ratio.max()
tbl['num_100_ratio_during_t_14-median'] = df_.groupby('msno').num_100_ratio.median()
tbl['num_100_ratio_during_t_14-std'] = df_.groupby('msno').num_100_ratio.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_100_ratio_during_t_14.csv'.format(folder), index = False)
del tbl
gc.collect()

##################################################
# n = 30
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 30).reset_index(drop = True)
#core1:completed_songs_ratio
print ('core1')
tbl = df_.groupby('msno').num_25_ratio.mean().to_frame()
tbl.columns = ['num_25_ratio_during_t_30-mean']
tbl['num_25_ratio_during_t_30-min'] = df_.groupby('msno').num_25_ratio.min()
tbl['num_25_ratio_during_t_30-max'] = df_.groupby('msno').num_25_ratio.max()
tbl['num_25_ratio_during_t_30-median'] = df_.groupby('msno').num_25_ratio.median()
tbl['num_25_ratio_during_t_30-std'] = df_.groupby('msno').num_25_ratio.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_25_ratio_during_t_30.csv'.format(folder), index = False)
del tbl
gc.collect()	
#core2:num_repeated_songs
print ('core2')
tbl = df_.groupby('msno').num_100_ratio.mean().to_frame()
tbl.columns = ['num_100_ratio_during_t_30-mean']
tbl['num_100_ratio_during_t_30-min'] = df_.groupby('msno').num_100_ratio.min()
tbl['num_100_ratio_during_t_30-max'] = df_.groupby('msno').num_100_ratio.max()
tbl['num_100_ratio_during_t_30-median'] = df_.groupby('msno').num_100_ratio.median()
tbl['num_100_ratio_during_t_30-std'] = df_.groupby('msno').num_100_ratio.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_100_ratio_during_t_30.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 60
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 60).reset_index(drop = True)
#core1:completed_songs_ratio
print ('core1')
tbl = df_.groupby('msno').num_25_ratio.mean().to_frame()
tbl.columns = ['num_25_ratio_during_t_60-mean']
tbl['num_25_ratio_during_t_60-min'] = df_.groupby('msno').num_25_ratio.min()
tbl['num_25_ratio_during_t_60-max'] = df_.groupby('msno').num_25_ratio.max()
tbl['num_25_ratio_during_t_60-median'] = df_.groupby('msno').num_25_ratio.median()
tbl['num_25_ratio_during_t_60-std'] = df_.groupby('msno').num_25_ratio.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_25_ratio_during_t_60.csv'.format(folder), index = False)
del tbl
gc.collect()	
#core2:num_repeated_songs
print ('core2')
tbl = df_.groupby('msno').num_100_ratio.mean().to_frame()
tbl.columns = ['num_100_ratio_during_t_60-mean']
tbl['num_100_ratio_during_t_60-min'] = df_.groupby('msno').num_100_ratio.min()
tbl['num_100_ratio_during_t_60-max'] = df_.groupby('msno').num_100_ratio.max()
tbl['num_100_ratio_during_t_60-median'] = df_.groupby('msno').num_100_ratio.median()
tbl['num_100_ratio_during_t_60-std'] = df_.groupby('msno').num_100_ratio.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_100_ratio_during_t_60.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 90
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 90).reset_index(drop = True)
#core1:completed_songs_ratio
print ('core1')
tbl = df_.groupby('msno').num_25_ratio.mean().to_frame()
tbl.columns = ['num_25_ratio_during_t_90-mean']
tbl['num_25_ratio_during_t_90-min'] = df_.groupby('msno').num_25_ratio.min()
tbl['num_25_ratio_during_t_90-max'] = df_.groupby('msno').num_25_ratio.max()
tbl['num_25_ratio_during_t_90-median'] = df_.groupby('msno').num_25_ratio.median()
tbl['num_25_ratio_during_t_90-std'] = df_.groupby('msno').num_25_ratio.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_25_ratio_during_t_90.csv'.format(folder), index = False)
del tbl
gc.collect()	
#core2:num_repeated_songs
print ('core2')
tbl = df_.groupby('msno').num_100_ratio.mean().to_frame()
tbl.columns = ['num_100_ratio_during_t_90-mean']
tbl['num_100_ratio_during_t_90-min'] = df_.groupby('msno').num_100_ratio.min()
tbl['num_100_ratio_during_t_90-max'] = df_.groupby('msno').num_100_ratio.max()
tbl['num_100_ratio_during_t_90-median'] = df_.groupby('msno').num_100_ratio.median()
tbl['num_100_ratio_during_t_90-std'] = df_.groupby('msno').num_100_ratio.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/num_100_ratio_during_t_90.csv'.format(folder), index = False)
del tbl
gc.collect()


100%|██████████| 6/6 [00:00<00:00, 1477.82it/s]

reduce memory
shape of df: (49445, 4)
core1
reduce memory



100%|██████████| 6/6 [00:00<00:00, 682.28it/s]


core2
reduce memory


100%|██████████| 6/6 [00:00<00:00, 1303.52it/s]

...

100%|██████████| 6/6 [00:00<00:00, 838.58it/s]

core1
reduce memory
core2
reduce memory



100%|██████████| 6/6 [00:00<00:00, 1276.61it/s]

core1
reduce memory
core2
reduce memory


80